### Könyvtárak importálása

In [17]:
import pandas as pd
import numpy as np
import os
ldir = os.chdir(r'C:\Users\Gergely\Documents\Datasets')

### Titanic dataset beolvasása

In [18]:
df = pd.read_csv('Titanic.csv')
df.head()

,Passengerid,Age,Fare,Sex,sibsp,Parch,Pclass,Embarked,Survived
0,1,22.0,7.2500,0,1,0,3,2.0,0
1,2,38.0,71.2833,1,1,0,1,0.0,1
2,3,26.0,7.9250,1,0,0,3,2.0,1
3,4,35.0,53.1000,1,1,0,1,2.0,1
4,5,35.0,8.0500,0,0,0,3,2.0,0


### Bemeneti változók (X) és célváltozó (y) meghatározása

In [19]:
X = df.iloc[:, 1:8].drop(axis=1, columns='Fare')
y = df.iloc[:, -1]
X.head()

,Age,Sex,sibsp,Parch,Pclass,Embarked
0,22.0,0,1,0,3,2.0
1,38.0,1,1,0,1,0.0
2,26.0,1,0,0,3,2.0
3,35.0,1,1,0,1,2.0
4,35.0,0,0,0,3,2.0


### Tanító és teszt halmaz készítés

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Tanítás

In [21]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression().fit(X_train, y_train)

### Predikció

In [22]:
y_predicted = logreg.predict(X_test)

### Confusion matrix (y_test vs y_predicted)

In [23]:
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_predicted)
cnf_matrix

array([[291,  17],
       [ 80,  44]], dtype=int64)

#### Metódusok

In [28]:
print('y lehetséges értékei: ', logreg.classes_)                            #classes_
print('accuracy (jól eltalált / összes) = ', logreg.score(X_test, y_test))  #score
print('b0: ', logreg.intercept_)                                            #intercept_ 
print('bn: ', logreg.coef_)                                                 #coef_      

y lehetséges értékei:  [0 1]
accuracy (jól eltalált / összes) =  0.7754629629629629
b0:  [0.67919689]
bn:  [[-0.02754952  1.66836759 -0.19098624 -0.01522924 -0.73342858 -0.03121808]]


### Stratified K-Fold cross validation

In [29]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5)

In [30]:
for train_index, test_index in kf.split(X, y):
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

In [31]:
logreg = LogisticRegression().fit(X_train, y_train)

In [32]:
y_predicted = logreg.predict(X_test)

In [33]:
cnf_matrix = metrics.confusion_matrix(y_test, y_predicted)
cnf_matrix

array([[154,  39],
       [ 34,  34]], dtype=int64)

In [34]:
logreg.score(X_test, y_test)

0.7203065134099617

### Logistic regression with cross validation (Stratified K-Fold)

In [35]:
from sklearn.linear_model import LogisticRegressionCV

logregcsv = LogisticRegressionCV(cv=5)
logregcsv.fit(X=X_train, y=y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [36]:
y_predictedcsv = logregcsv.predict(X_test)

In [37]:
cnf_matrix = metrics.confusion_matrix(y_test, y_predictedcsv)
cnf_matrix

array([[153,  40],
       [ 34,  34]], dtype=int64)

In [38]:
logregcsv.score(X_test, y_test)

0.7164750957854407

In [39]:
#kor, nem (0-férfi, 1-nő), testvérek és házastársak száma a hajón, szülők és gyermekek száma a hajón, utazás osztálya, felszállás helye
anna=[1, 0, 1, 1, 1, 2]
esely=logreg.predict_proba(np.array(anna).reshape(1,-1))[:,1]
tuleles=logreg.predict(np.array(anna).reshape(1,-1))
if tuleles==1:
    print("Túlélte! Ennyi esélye volt rá: ", esely)
else:
    print("Sajnos nem élte túl :c Ennyi esélye volt:", esely)

Sajnos nem élte túl :c Ennyi esélye volt: [0.42415578]
